<a href="https://colab.research.google.com/github/pulavartivinay/Face_Mask_Detection/blob/model/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# start

In [ ]:
# installs

In [ ]:
# imports

In [15]:
# loading dataset
# https://www.kaggle.com/ashishjangra27/face-mask-12k-images-dataset

!gdown --id 1NOzYPR3zAS8e2EFdRvjYpSBxg9RaDfxe #loading the dataset from google drive link to colab
!unzip /content/face-mask-12k.zip &> /dev/null #unzipping
!rm -rf /content/face-mask-12k.zip &> /dev/null #deleting the zip file
!rm -rf __MACOSX &> /dev/null

Downloading...
From: https://drive.google.com/uc?id=1NOzYPR3zAS8e2EFdRvjYpSBxg9RaDfxe
To: /content/face-mask-12k.zip
351MB [00:02, 165MB/s]


In [ ]:
# Preprocessing the data set into list of images -- Vinay
# https://keras.io/api/preprocessing/image/#image_dataset_from_directory-function

In [ ]:
# Model(Transfer Learning) (optional)

In [ ]:
# Main Model 

In [ ]:
# Setting the Hyper Parameters

In [ ]:
# Model Compilation -- Vinay
# https://keras.io/api/models/model_training_apis/

In [ ]:
# Training -- Vipul
# https://keras.io/api/models/model_training_apis/

In [ ]:
# Adding call Back Functions(optional)
# https://medium.com/@thongonary/how-to-compute-f1-score-for-each-epoch-in-keras-a1acd17715a2

In [ ]:
# model evaluation
# https://keras.io/api/models/model_training_apis/

In [ ]:
# Predictions of single random image -- Vipul
# https://keras.io/api/models/model_training_apis/

In [ ]:
# Drawing the corresponding boundary of single random image on parent random image -- Vinay

In [ ]:
# Predictions of Validation data set(Dev) -- Vinay

In [ ]:
# Drawing the corresponding boundary of some random images on parent images from Dev data set -- Vipul

In [ ]:
# Drawing the Loss and accuracy of the model -- Vipul

In [ ]:
# Calculating the F1-Score of the Model on Dev data set -- Vinay
# https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
# https://medium.com/@thongonary/how-to-compute-f1-score-for-each-epoch-in-keras-a1acd17715a2

In [ ]:
# Visualisation of the F1-Score -- Vinay